# I. DATA PREPARATION

In [122]:
# we will import all the necessary libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import json
import re
from shapely.geometry import Point, LineString #this library is for manipulating geometric objects, and it is what geopandas uses to store geometries
from scipy.spatial import distance

In [123]:
#tlf provides passenger flow data for each network separately
#we need to create a list of all the different files we need
df_dlr = pd.read_csv('https://www.dropbox.com/scl/fi/xkf9ttbop2wyypp8bttgw/NBT19MTT2b_od__DLR_tb_wf.csv?rlkey=me22ozynt7i13ah9hvh9f157m&dl=1')
df_ezl = pd.read_csv('https://www.dropbox.com/scl/fi/vizzmmz5u1mhhxignbwei/NBT19MTT2b_od__EZL_tb_wf.csv?rlkey=nz69dj4n7x02389gl7gc95knw&dl=1')
df_lo = pd.read_csv('https://www.dropbox.com/scl/fi/rhofj793lzlh6ptheo0ui/NBT19MTT2b_od__LO_tb_wf.csv?rlkey=btj1o5i02dx9paqhpkhk1chlo&dl=1')
df_lu = pd.read_csv('https://www.dropbox.com/scl/fi/572yutkebg3pa4pjn42d9/NBT19MTT2b_od__LU_tb_wf.csv?rlkey=0rfklc4u7nxcwbtsjsko0i3xd&dl=1')

files = [df_dlr, df_ezl, df_lo, df_lu]
london_OD = pd.concat(files, ignore_index=True)

In [124]:
london_OD

mode_mnlc_o  mode_mnlc_d          2         3        4        5  \
0              504          523    8.03300    16.430   24.563   28.555   
1              504          533   30.71900    40.105   61.571   57.881   
2              504          538    5.94100    17.238   37.493   43.158   
3              504          559    5.79500    26.397   27.285   24.065   
4              504          573   77.78600   113.750  143.938  387.562   
...            ...          ...        ...       ...      ...      ...   
62575          884          784   97.99600   538.180  298.849  173.856   
62576          884          787  115.91900   281.952  149.409  138.607   
62577          884          788  139.04400   682.340  768.051  504.189   
62578          884          789  156.23601   481.302  702.600  595.686   
62579          884          852  283.30700  1191.298  809.155  366.224   

             6        7 mode    1    8  
0       13.023    2.263  DLR  NaN  NaN  
1       15.573    0.317  DLR  NaN  NaN  
2       33.890   10.429  DLR  NaN  NaN  
3       15.213    8.508  DLR  NaN  NaN  
4      165.776   65.283  DLR  NaN  NaN  
...        ...      ...  ...  ...  ...  
62575   51.263   14.653   LU  0.0  0.0  
62576   64.885   25.433   LU  0.0  0.0  
62577  345.944  113.663   LU  0.0  0.0  
62578  317.113   63.273   LU  0.0  0.0  
62579  177.616   34.625   LU  0.0  0.0  

[62580 rows x 11 columns]

In [125]:
definition_df = pd.read_excel('https://www.dropbox.com/scl/fi/8rvd82qd4ald2yyt2bz6m/NBT19_Definitions.xlsx?rlkey=5jcia5kufnlmictzcs9nc2qia&dl=1', 'Stations')

In [126]:
#we will drop any missing values from our dataframe
definition_df.dropna(inplace=True)

#transform mnlc codes from floats to int
definition_df.MNLC = definition_df.MNLC.astype(int)

In [127]:
#add the station names
london_OD['station_origin'] = london_OD['mode_mnlc_o'].apply(lambda x: definition_df[definition_df.MNLC==x]['StationName'].values[0])
london_OD['station_destination'] = london_OD['mode_mnlc_d'].apply(lambda x: definition_df[definition_df.MNLC==x]['StationName'].values[0])

KeyboardInterrupt: 

In [ ]:
# we will only keep passenger flows for the timeband we are interested in
london_OD_AMpeak = london_OD[['station_origin', 'station_destination', '3']].copy()

In [ ]:
#rename column
london_OD_AMpeak.rename({'3':'flows'}, axis=1, inplace=True)

#since the flows are averages the are stored as flows.
#for our analysis we will turn the into ints
london_OD_AMpeak.flows = london_OD_AMpeak.flows.astype(int)

In [ ]:
station_origin = list(london_OD_AMpeak.station_origin.values)
station_destination = list(london_OD_AMpeak.station_destination.values)
all_stations = list(set(station_origin + station_destination))
all_stations = sorted(all_stations, key=str.lower)

In [ ]:
london_OD_AMpeak['station_origin']= london_OD_AMpeak.station_origin.apply(lambda x: 
                                      re.sub('\sLU\s?|\sLO\s?|\sNR\s?|\sTf[lL]\s?|\sDLR\s?|\s\(.*\)', '', x)
                                     )
london_OD_AMpeak['station_destination']=london_OD_AMpeak.station_destination.apply(lambda x: 
                                      re.sub('\sLU\s?|\sLO\s?|\sNR\s?|\sTf[lL]\s?|\sDLR\s?|\s\(.*\)', '', x)
                                     )

In [ ]:
#group by origin and destination station and sum the flows
london_OD_AMpeak = london_OD_AMpeak.groupby(['station_origin', 'station_destination'], as_index=False).sum()

In [ ]:
#let's sanity check and see the highest and lowest flows in and out flows
outflows = london_OD_AMpeak.groupby('station_origin', as_index=False).sum()
#let's sanity check and see the highest and lowest flows in and out flows
inflows = london_OD_AMpeak.groupby('station_destination', as_index=False).sum()

### Load London Underground shapefile data

In [ ]:
#read in the data using geopandas
stations = gpd.read_file('https://www.dropbox.com/scl/fi/j1ugr3bsub9jhp8b0btkq/tfl_stations.json?rlkey=r38t2xn9n5erc1y3jzub6tqol&dl=1')
lines = gpd.read_file('https://www.dropbox.com/scl/fi/ov98qi02qju6kcq5vxnbb/tfl_lines.json?rlkey=h9wip7f1drqn4prgf0fhkcvna&dl=1')

In [ ]:
stations

In [ ]:
#project data to British National Grid
#we do this so we can work in meters
stations = stations.to_crs(epsg = 27700)
lines = lines.to_crs(epsg = 27700)

### Clean station names

We have multiple points representing one station (for example Paddington has seperate points for different entrances). We will simplify this by using RegEx make the names consistent. 

Once we have cleaned the names we can set the coordinate of these stations to the mean values of the coordinates of all the stations with the same name.

In [ ]:
#get all station names
station_names = {row.id: row['name'] for i, row in stations.iterrows()}

#clean station names
for k, v in station_names.items():
    if k == 'none':
        continue
    cleaned_name = re.sub('\sLU\s?|\sLO\s?|\sNR\s?|\sTf[lL]\s?|\sDLR\s?|\s\(.*\)', '', v)
    
    #note that bank and monument are enconded as one station in our TFL data, so we will do the same
    if cleaned_name in ['Bank', 'Monument']:
        cleaned_name = 'Bank and Monument'
    station_names[k] = cleaned_name
    
stations['name'] = stations['id'].apply(lambda x: station_names[x])

#stations[['name','id']].groupby('name').count().sort_values(by='id',ascending=False)

**Change position of station to mean of all station with the same name**

In [ ]:
#get position of stations as mean of x, y of stations

#create empty dict to store list of coordinates
pos = {}

#iterate through the stations
for i, station in stations.iterrows():
    xy = station.geometry.coords[0]
    
    #if station already in dict add coordinate to coordinate list
    if station['name'] in pos.keys():
        pos[station['name']].extend([xy])
    #if station is not in the dict add station to dict and set first coordinate in list
    else:
        pos[station['name']] = [xy]

#iterate through our dict and replace coordinate list with mean value
for k, v in pos.items():
    #we set axis to 0 to make sure to take mean of x and y coordinates
    pos[k] = np.mean(v, axis=0)

In [ ]:
#set new geometry
stations['geometry'] = stations['name'].apply(lambda x: Point(pos[x]))

In [ ]:
stations

### Lines file

In [ ]:
#get all line types as a list and subset geoms by lines
line_names  = lines.lines.apply(lambda x: [x['name'] for x in json.loads(x)] )
line_names = list(set([item for sublist in line_names for item in sublist]))

In [ ]:
#We can see that east london line is incomplete
#digging into this however we notice that it's because east london is actually part of the overground
#merge East London line and London Overground since they are one line
lines.lines = lines.lines.str.replace('East London', 'London Overground')
stations.lines = stations.lines.str.replace('East London', 'London Overground')

In [ ]:
#Our TFL data doesn't include tramlink, emirates air lines, and crossrail is still in construction (except not really)
#exclude the lines that we are not going to use
excluded_lines = ['Thameslink 6tph line', 'East London', 'Crossrail 2', 'Emirates Air Line', 'Crossrail', 'Tramlink']

In [ ]:
stations

## Create seperate graphs for each line and then join

Since we don't have properly constructed line geometries because:
* there are discontinuities in the line geometries
* a line between two stations can actually be composed of more than one geometry

We will construct geometric graphs (meaning graphs just from the line geometries where nodes are coordinates that define the line geometry). It's easy to fix the discontinuities in this manner because the can be thought of as nodes with degree = 1 that have another node with degree = 1 within a threshold distance. 

Once we have these discontinuties fixed we can use these geometric graphs to construct proper geometries between stations by taking the shortest paths within these networks starting from the node closest to the origin station and ending in the node closest to the destination station. The shortest path will then be a list of coordinates that define the line geometry between the two stations. 

In [ ]:
#create an empty dict to hold our geometric graphs
geometric_graphs = {}

#iterate through each line individually
for line_name in line_names:
    #skip lines that we won't be using
    if line_name in excluded_lines :
        continue
    print(line_name)
    
    #subset our line and station files for the ones the belong to a specific line
    temp_lines = lines[lines.lines.str.contains(line_name)]
    temp_stations = stations[stations.lines.str.contains(line_name)]
    
    #create empty graph
    G= nx.Graph()
    
    #iterate through our line geometries
    for i, line in temp_lines.iterrows():
        #get list of coordinates that define our line
        _l = list(line.geometry.coords)

        #add coordinates as edges
        G.add_edges_from(list(zip(_l,_l[1:])))
    
    #remove any self loops
    G.remove_edges_from(nx.selfloop_edges(G))
    
    #the position of the node is the name of the node
    #let's extract this to plot our graph
    pos = {n: n for n in G.nodes()}
    
    # get degree one nodes
    D = nx.degree(G)
    k1 = [node for node,degree in D if degree==1]
    #let's plot our graph and all degree one nodes
    nx.draw(G, pos=pos, node_size=0.5, node_color='grey')
    nx.draw_networkx_nodes(G, pos=pos, nodelist=k1, node_color='red', node_size=8)
    plt.show()
    
    #get distance between all points of 1-degree, this will be a matrix
    dist = distance.cdist(k1, k1)
    
    #add edges between points that are close
    for j in range(dist.shape[0]-1):
        temp = dist[j][j+1:]
        #get index of closest node
        i_min = np.argmin(temp)
        if dist[j][i_min+j+1]<50:
            G.add_edge(k1[j],k1[i_min+j+1])
    
    #recalculate degree
    D = nx.degree(G)
    k1 = [node for node,degree in D if degree==1]
    #plot new graph
    nx.draw(G, pos=pos, node_size=0.5, node_color='grey')
    nx.draw_networkx_nodes(G, pos=pos, nodelist=k1, node_color='red', node_size=8)
    plt.show()
    print(nx.number_connected_components(G))
    #store graph in dictionary
    geometric_graphs[line_name] = G


Now that we have these geometric graphs we can create our final graph by using the data in the lines attribute in our lines geodataframe that contains origin and destination station. We can then use our geometric graph and find the shortest route to create our line geometries and find the length of the lines. 

We will store this information in a dataframe that we can use to create our final graph

In [ ]:
#create a dataframe with all geometries from all valid lines
#this list will contain a dictionary for each edge in our graph that contains
# node pairs, line name, and a geometry (this is just for quick visualization purposes and will be replaced by true geometry)
edge_list = []
def _has_ids(k):
    #helper function to check if line contains id of start and end station
    if ('start_sid' in k.keys()) and ('end_sid' in k.keys()):
        return True
    else:
        return False

#iterate through each line seperately 
for line_name in line_names:
    if line_name in excluded_lines :
        continue
        
    #subset lines and stations
    temp_lines = lines[lines.lines.str.contains(line_name)]
    temp_stations = stations[stations.lines.str.contains(line_name)]
    
    #for each line in our subset dataframe get each origin and destination pair
    for i, line in temp_lines.iterrows():
        for k in json.loads(line.lines):
            if k['name'] == line_name:
                if _has_ids(k):
                    #get start and end station ids
                    start_id = k['start_sid']
                    end_id = k['end_sid']

                    #get info of start and end station
                    start_station = temp_stations[(temp_stations.id == start_id) | (temp_stations.altmodeid == start_id)].values
                    end_station = temp_stations[(temp_stations.id == end_id) | (temp_stations.altmodeid == end_id)].values
                    
                    #if no matching station exist, let's just take the starting and ending coordinate of the line for now
                    if len(start_station) >= 1 and len(end_station) >=1:
                        s_geom = start_station[0][-1].coords[0]
                        e_geom = end_station[0][-1].coords[0]
                        edge_list.append({
                            'line_name': line_name,
                            'start_id': start_id,
                            'end_id': end_id,
                            'geometry': LineString([s_geom, e_geom])
                        })

In [ ]:
#turn our dictionary list into a geopandas df to quickly plot and sanity check our work
edge_gdf = gpd.GeoDataFrame(edge_list)
edge_gdf.crs = lines.crs

In [ ]:
### Add geometry information back into the graph
for line_name in line_names:
    if line_name in excluded_lines :
        continue
    temp_lines = edge_gdf[edge_gdf.line_name == line_name]
    temp_stations = stations[stations.lines.str.contains(line_name)]
    
    #let's plot our lines so we can check them
    fig, ax = plt.subplots(figsize = (7,7))
    fig.suptitle(f'line: {line_name}')
    ax.set_aspect("equal")
    ax.axis('off')
    ax.margins(0.1)
    
    base = temp_lines.plot(ax=ax, color='grey')
    temp_stations.plot(ax=base, markersize=20, color='red')
    plt.show()
    
    #add real geometry back
    for i, edge in temp_lines.iterrows():
        #get start and end of line coordinates
        start = edge.geometry.coords[0]
        end = edge.geometry.coords[-1]
        
        #let's get all the coordinates in our geometric graph
        nodes = list(geometric_graphs[line_name].nodes)
        
        #find nearest node in graph to origin and destination
        s_dist = distance.cdist([start], nodes)[0]
        s_i = np.argmin(s_dist)
        source= nodes[s_i]
        
        t_dist = distance.cdist([end], nodes)[0]
        t_i = np.argmin(t_dist)
        target= nodes[t_i]
        
        #get shortest path
        sp = nx.shortest_path(geometric_graphs[line_name], source, target)
        
        #make into geometry
        #notice how I'm also adding the start and end coordinates to the line definition
        #this fixes the problem of stations and lines not matching up
        geometry = LineString([start] + sp + [end])
        edge_gdf.loc[i, 'geometry'] = geometry

In [ ]:
#let's store the length of the real geometry
edge_gdf['length'] = edge_gdf.geometry.length

In [ ]:
#let's replace the ids with the actual name
#get all station names
station_names_2 = {row.altmodeid: row['name'] for i, row in stations.iterrows() if row.altmodeid != None}
station_names.update(station_names_2)

In [ ]:
edge_gdf['start_id'] = edge_gdf['start_id'].apply(lambda x: station_names[x])
edge_gdf['end_id'] = edge_gdf['end_id'].apply(lambda x: station_names[x])

## IMPORTANT!!!

In [ ]:
#now we can use this to create our network!
G = nx.from_pandas_edgelist(edge_gdf, source = 'start_id', target='end_id', edge_attr=['length', 'line_name'])

#let's plot
#notice that even though we created the real geometries, the graph still plots just straight line
#this is because networkx doesn't have a concept of a edge geometry
#however this is ok, since we only need the distance as weights, and that is already an attribute in our graph
pos = {row['name']: row.geometry.coords[0] for i, row in stations.iterrows() if row['name'] in G.nodes()}
Gcc = nx.connected_components(G)
for n in Gcc:
    G_sub = G.subgraph(n)
    lines = [data['line_name'] for u,v, data in G_sub.edges(data=True)]
    print(set(lines))
    nx.draw(G_sub, pos, node_size=4)
    plt.show()

#save position to graph
nx.set_node_attributes(G, pos, 'coords')

### Sanity check our network

In [ ]:
#check that all distances are greater or equal to euclidean distance
for u,v, data in G.edges(data=True):
    assert(data['length'] >= distance.euclidean(pos[u], pos[v]))

In [ ]:
#if want to know what is the maximum distance between stations
max(dict(G.edges).items(), key=lambda x: x[1]['length'])

In [ ]:
#if want to know to which stations Baker Street is directly connected to
Baker_Street = [(u,v) for  u,v in G.edges() if u == 'Baker Street' or v == 'Baker Street']
Baker_Street

In [ ]:
#We can also check the degree of the nodes in our network and check that they make sense
deg_london = nx.degree(G)

In [ ]:
df = pd.DataFrame(index = dict(deg_london).keys())
df['degree'] = dict(deg_london).values()

In [ ]:
df.sort_values('degree', ascending =False).head(20)

### Merge Network with OD data

ok, we are almost done!

We have processed our raw data, now we need to combine the two (TLF OD and our Network). For this we need two types of merge:

1. add flows as weights to the network (to be able to calculate disruptions to the network)
2. create OD with distance for our spatial interaction models (for our spatial interaction models)

In [ ]:
#check that our OD data and network data match
OD_names = set(london_OD_AMpeak.station_origin.unique())
network_names = set([n for n in G.nodes()])

Ok, we have a few stations that don't match up. They are small enough that we can solve this quickly by creating a mapping between names of the stations that we can keep.

* Battersea power station and Nine Elms data doesn't exist in our TFL data because the station wasn't opened when the data was collected. This was an extension of Northern line that opened recently.
* Action Main Line, Hanwell, Hayes & Harlington, Southall, and West Ealing are part of crossrail which we are not considering

In [ ]:
name_map = {
    'Heathrow Terminal 4 EL': 'Heathrow Terminal 4',
    'Heathrow Terminals 123': 'Heathrow Terminals 2 & 3',
    'Heathrow Terminals 2 & 3 EL': 'Heathrow Terminals 2 & 3',
    "Walthamstow Queen's Road": 'Walthamstow Queens Road'
}

In [ ]:
london_OD_AMpeak.station_origin = london_OD_AMpeak.station_origin.apply(
    lambda x: name_map[x] if x in name_map.keys() else x
)
london_OD_AMpeak.station_destination = london_OD_AMpeak.station_destination.apply(
    lambda x: name_map[x] if x in name_map.keys() else x
)

In [ ]:
#let's filter out the stations that we don't have in our network
OD_names = set(london_OD_AMpeak.station_origin.unique())
_filter = list(network_names.symmetric_difference(OD_names))
_filter

In [ ]:
london_OD_AMpeak = london_OD_AMpeak[~london_OD_AMpeak.station_origin.isin(_filter)]
london_OD_AMpeak = london_OD_AMpeak[~london_OD_AMpeak.station_destination.isin(_filter)]

**Add flow data to our network**

Our TFL data contains flows for OD pairs, but we don't know the flows passing through each edge in our network. We will have to calculate this assuming fall people travelling from Origin to Destination station are taking the shortest path within the network.

In [ ]:
#create a dictionary to store flows for all edges
flows = {(u,v): 0 for u,v in G.edges()}

In [ ]:
#calculate shortest paths for all flows and add data to dict
for i, row in london_OD_AMpeak.iterrows():
    source = row.station_origin
    target = row.station_destination
    
    #get shortest path
    path = nx.dijkstra_path(G, source, target)
    
    #our path is a list of nodes, we need to turn this to a list of edges
    path_edges = list(zip(path,path[1:])) 
    
    #add flows to our dict
    for u,v in path_edges:
        try:
            flows[(u,v)] += row.flows
        except:
            flows[(v,u)] += row.flows

### IMPORTANT!!! (INTEGRATION BETWEEN NETWORK AND WEIGHT)

In [ ]:
#assumption, uniform speed x different distance = time
#time = distance / speed
#speed = 33kmph or 9mps



In [ ]:
for edge, data in dict(G.edges).items():
    print(f"Edge: {edge}, Data: {data}")

In [ ]:
# Add 'velocity' field to each edge with value 60
for u, v, data in G.edges(data=True):
    data['velocity'] = 9

In [ ]:
for edge, data in dict(G.edges).items():
    print(f"Edge: {edge}, Data: {data}")

In [ ]:
# Add 'velocity' field to each edge with value 60
for u, v, data in G.edges(data=True):
    data['train_time'] = data['length'] / data['velocity']

In [ ]:
# Add 'velocity' field to each edge with value 60
for u, v, data in G.edges(data=True):
    data['dwell_time'] = data['flows'] / 50000 * 120
    data['travel_time'] = data['dwell_time'] + data['train_time']

In [ ]:
#creating the new weight, time and flows (travel time)
#travel_time = f(flows, train_time, dwell_time) # finding the formula
#travel_time = train_time + (flows * dwell time)

# Estimating Additional Dwell Time for 1,000 Passengers:

# If we assume a boarding/alighting time of 2.5 seconds per person (an average in crowded conditions)
#dwell_time = 1,000 people * 2.5 seconds/person = 2,500 seconds.
    
# Considering Multiple Doors
# London Underground trains have multiple doors per carriage (typically 3 to 4 doors per side, with about 6 to 8 carriages)
#dwell_time = Total boarding time / Number of doors = 2,500 seconds / 20 = 125 seconds (~2 minus for every additional 1000 passengers)

#travel_time (A->B) = train_time (A->B) + ((total flows of all stations between A->B /1000) * 2 min)


In [ ]:
#inputting the new weight into the network
#set this as a network attribute (change flow to new weight)
#nx.set_edge_attributes(G, flows, 'flows') # <- change into travel_time

#plot our calcuated flows
max_travel_time = max(data['travel_time'] for u, v, data in G.edges(data=True))
travel_color = [(i[2]['travel_time']/max_travel_time) for i in G.edges(data=True)]
travel_width = [(i[2]['travel_time']/max_travel_time)*10 for i in G.edges(data=True)]

# Plot graph
fig, ax = plt.subplots(figsize=(12,12))

#pos=nx.spring_layout(X)
edg=nx.draw_networkx_edges(G, pos,edge_color=travel_color, width=travel_width)

nx.draw_networkx_nodes(G,
        pos = pos,
        node_color= 'black',
        node_size= 1)

plt.colorbar(edg,label="Passenger Travel Time",orientation="horizontal", shrink=0.5)
plt.axis("off")
plt.title("London network Passenger Travel Time",fontsize=15)
plt.show()

In [ ]:
#set this as a network attribute (change flow to new weight)
nx.set_edge_attributes(G, flows, 'flows') # <- change into travel_time

#plot our calcuated flows
flows_values = flows.values()
flow_color=[(i[2]['flows']/max(flows_values)) for i in G.edges(data=True)]
flow_width=[(i[2]['flows']/max(flows_values)*10) for i in G.edges(data=True)]

# Plot graph
fig, ax = plt.subplots(figsize=(12,12))

#pos=nx.spring_layout(X)
edg=nx.draw_networkx_edges(G, pos,edge_color=flow_color, width=flow_width)

nx.draw_networkx_nodes(G,
        pos = pos,
        node_color= 'black',
        node_size= 1)

plt.colorbar(edg,label="Passenger Flows",orientation="horizontal", shrink=0.5)
plt.axis("off")
plt.title("London network Passenger Flows",fontsize=15)
plt.show()

In [ ]:
# weighted alternatives
## 1. time and flow = weight


# II. Average Shortest Path Length (APL) Analysis

In [ ]:
## a. Scenario 1: Business As Usual

In [ ]:
avg_shortest_path_length = nx.average_shortest_path_length(G, weight='travel_time')
print(avg_shortest_path_length)

In [ ]:
# Calculate all-pairs shortest paths (weighted)
path_lengths = dict(nx.all_pairs_dijkstra_path_length(G, weight='travel_time'))

# Extract the shortest path lengths into a list
lengths = []
for source, target_dict in path_lengths.items():
    for target, length in target_dict.items():
        if source != target:  # avoid zero-length paths (self loops)
            lengths.append(length)

# Plot histogram
plt.hist(lengths, bins=5, edgecolor='black')
plt.title('Histogram of Shortest Path Lengths (Weighted)')
plt.xlabel('Path Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
## b. Scenario 2: Winter Disruption without Government Intervention

In [ ]:
## c. Scenario 3: Winter Disruption with Government Intervention

In [ ]:
## d. APL Comparison between Scenarios

# III. Betweeness Centrality (BC) Analysis

In [ ]:
## a. Scenario 1: Business As Usual

In [ ]:
## b. Scenario 2: Winter Disruption without Government Intervention

In [ ]:
## c. Scenario 3: Winter Disruption with Government Intervention

In [ ]:
## d. BC Comparison between Scenarios

# IV. Intervention Recommendation